# RFSoC Spectrum Sweep
----
<div class="alert alert-box alert-info">
Please use Jupyter labs http://board_ip_address/lab for this notebook.
</div>

A simple way of validating the operation of the RF Data Converters
(RF DCs) on your RFSoC4x2 board is to perform a spectrum sweep. 
The spectrum sweep is the process of transmitting a series of
tones and confirming that they are all received correctly.

## Aims
* Review basic tone generation using the RF DAC channels.
* Demonstrate a simple Spectrum Analyzer using the RF ADC channels.
* Use the tone generator and Spectrum Analyzer to perform a spectrum sweep.

## Table of Contents
* [Hardware Setup](#hardware-setup)
* [Application Setup and Design](#application-setup-and-design)
    * [Basic Tone Generation](#basic-tone-generation)
    * [A Spectrum Analyzer](#a-spectrum-analyser)
* [The Spectrum Sweep](#the-spectrum-sweep)
    * [Setting Constraints](#setting-constraints)
    * [Useful Measurements](#useful-measurements)
    * [Running the Sweep](#running-the-sweep)
    * [Analysing Results](#analysing-results)
* [Conclusion](#conclusion)

## References
* [Xilinx, Inc, "USP RF Data Converter: LogiCORE IP Product Guide", PG269, v2.4, November 2020](https://www.xilinx.com/support/documentation/ip_documentation/usp_rf_data_converter/v2_4/pg269-rf-data-converter.pdf)
* [Cooley, James W.; Tukey, John W. (1965). "An algorithm for the machine calculation of complex Fourier series". Mathematics of Computation. 19 (90): 297–301.](https://www.ams.org/journals/mcom/1965-19-090/S0025-5718-1965-0178586-1/home.html)

## Revision History
* v1.0 | 25/02/2022 | First notebook revision.

----

## Hardware Setup <a class="anchor" id="hardware-setup"></a>

The RFSoC4x2 board contains 4 RF ADC channels and 2 RF DAC channels. For this demonstration, we will be using 2 RF ADC channels and 2 RF DAC channels in loopback.

Make the loopback connection, using SMA cables, as shown below:

* Channel 3: DAC A to ADC A
* Channel 2: DAC B to ADC B

See the image below for a demonstration.

<img src='data/rfsoc4x2_loopback.jpg' align='left' style='left' height='75%' width='75%'/>

<div class="alert alert-heading alert-danger">
    <h4 class="alert-heading">Warning:</h4>

In this demo we are transmitting signals via cables.
This device can also transmit wireless signals. 
Keep in mind that unlicensed transmission of wireless signals
may be illegal in your location. 
Radio signals may also interfere with nearby devices,
such as pacemakers and emergency radio equipment. 
If you are unsure, please seek professional support.
</div>

----

## Application Setup and Design <a class="anchor" id="application-setup-and-design"></a>

The spectrum sweep application needs two major components to operate effectively.
A tone generator that creates frequency stimulus, and a spectrum analysis
tool such as a Spectrum Analyzer. We will design and setup each of these
components and then perform the spectrum sweep application.

To begin we will download the base overlay.

In [1]:
from pynq.overlays.base import BaseOverlay

base = BaseOverlay('base.bit')

The RFSoC4x2 has a sophisticated clocking network, which can generate
low-jitter clocks for the RF DC Phase-Locked Loops (PLLs).

The base overlay has a simple method to initialize these clocks. Run the cell below to set the LMK and LMX clocks to 245.76MHz and 491.52MHz, respectively.

In [2]:
base.init_rf_clks()

### Basic Tone Generation <a class="anchor" id="basic-tone-generation"></a>
Tone generation has been covered before in previous notebooks, however, 
we require it again to perform the spectrum sweep application. 

Lets first obtain the ToneGenerator class from `rfsystem.spectrum_sweep` and
create a ToneGenerator for each channel in the system.

In [3]:
from rfsystem.spectrum_sweep import ToneGenerator

generators = []
for i in range(0, len(base.radio.transmitter.channel)):
    generators.append(
        ToneGenerator(channel=base.radio.transmitter.channel[i]))

Now we can initialise each tone generator with a centre frequency and gain.

In [4]:
for generator in generators:
    generator.frequency_selector.centre_frequency = 600 # MHz
    generator.amplitude_controller.enable = True
    generator.amplitude_controller.gain = 0.5

Although the RF DAC can produce a tone of a desired center frequency,
one should keep in mind that the tone folds through the RF DAC Nyquist Zones.
While you may be transmitting a 600 MHz tone, as in the above code cell, 
you are also transmitting another tone at 4315.2 MHz (due to a sample frequency
of 4915.2 Msps). External RF filtering is required to suppress unwanted tones
in neighboring Nyquist Zones.

### A Spectrum Analyzer <a class="anchor" id="a-spectrum-analyser"></a>

A simple method of inspecting a signal is to perform an FFT and
analyze it in the frequency domain. A Spectrum Analyzer class has been
created for this very purpose in `rfsystem.spectrum_sweep`. Let's import it
and create a spectrum analyzer for each receiver channel.

In [5]:
import ipywidgets as ipw
from rfsystem.spectrum_sweep import SpectrumAnalyser

analysers = []
sample_frequency = 2457.6e6
number_samples = 12288
for i in [2,3]:
    analysers.append(
        SpectrumAnalyser(channel = base.radio.receiver.channel[i],
                         sample_frequency = sample_frequency,
                         number_samples = number_samples,
                         title = ''.join(['Spectrum Analyser Channel ',
                                          str(i)]),
                         height = None,
                         width = None))

ipw.VBox([analyser.spectrum_plot.get_plot() for analyser in analysers])

    'data': [{'name': 'Spectrum',
              'type': 'scatter',
             …

You will now see two spectrum plots on your screen. We can animate these plots by running the code cell below.

In [6]:
import time

for i in range(0, 20):
    for analyser in analysers:
        analyser.update_spectrum()
    time.sleep(0.2)

Excellent! You have just explored a software defined spectrum analyzer. 
It will be useful if you right click the above plots and select
`Create new view for output`. This will give the spectrum analyzers their own
window. You will be able to see them at all times when scrolling through the
remainder of this notebook.

## The Spectrum Sweep <a class="anchor" id="the-spectrum-sweep"></a>

We have created a tone generator and spectrum analyzer for each channel.
The next stage is to perform the spectrum sweep and validate the operation of
the RF DCs.

### Setting Constraints <a class="anchor" id="setting-constraints"></a>
We need to define the boundaries of our spectrum sweep application.
What frequencies will we test? At which power should the tone generator
operate at? We can set these parameters below.

In [7]:
frequency_list = [200, 600, 1000, 1600, 2600, 3200, 3600, 4200, 4400, 4800]
transmitter_amplitude = 0.5

### Useful Measurements <a class="anchor" id="useful-measurements"></a>

We should consider the measurements during the spectrum sweep. 

* Signal to Noise Ratio (the different between the peak signal and noise). 
* The fundamental frequency of the received spectrum.

Both of the above measurements are defined in the methods below.

In [8]:
import numpy as np

def calculate_simple_snr(spectrum):
    return np.max(spectrum) - np.average(spectrum)

def calculate_fundamental(spectrum, sample_frequency, number_samples, 
                          receive_frequency):
    rbw = sample_frequency/number_samples
    maxindex = np.argsort(spectrum)[-1:][0]
    return maxindex * rbw + receive_frequency - sample_frequency/2

### Running the Sweep <a class="anchor" id="running-the-sweep"></a>

To run the sweep, we need to iterate through all of the frequencies in
`frequency_list` at least once for each channel. The tone generator will
need to be set to the appropriate frequency and the spectrum analyzer needs
to be in the correct Nyquist Zone. 

We have created the spectrum sweep below.

In [9]:
import time

def run_spectrum_sweep(frequencies = frequency_list,
                       amplitude = transmitter_amplitude):
    fundamental = []
    snr = []
    results = []
    frequency_zones = [1228.8, 3686.4]
    for frequency in frequencies:
        for generator in generators:
            generator.amplitude_controller.enable = True
            generator.amplitude_controller.gain = amplitude
            generator.frequency_selector.centre_frequency = frequency

        time.sleep(1)

        for analyser in analysers:
            if frequency*1e6 < sample_frequency:
                receive_frequency = frequency_zones[0]
            else:
                receive_frequency = frequency_zones[1]
    
            analyser.frequency_selector.centre_frequency = receive_frequency
            analyser.spectrum_plot.centre_frequency = receive_frequency*1e6
            analyser.update_spectrum()
            spectrum = analyser.get_spectrum()
            maxvalue = np.max(spectrum)
            maxindex = np.argsort(spectrum)[-1:][0]
            fundamental.append(
                calculate_fundamental(
                    spectrum, sample_frequency, number_samples, 
                    receive_frequency*1e6)*1e-6)
            snr.append(calculate_simple_snr(spectrum))

    for i in range(0, len(analysers)):
        results.append(
            np.array([frequencies, fundamental[i::len(analysers)], 
                      snr[i::len(analysers)]]))

    return results

Running the following code cell will execute the spectrum sweep application.
It will take some time to complete. 

While you wait, you can inspect the spectrum analyzer plots.

In [10]:
results = run_spectrum_sweep()

### Analysing Results <a class="anchor" id="analysing-results"></a>

Previously we had defined our measurements to be the SNR, and the fundamental
frequency of the received signal. Let's inspect these measurements now. 

In [11]:
import pandas as pd

for i in range(len(analysers)):
    print(''.join(['Channel: ', str(i+2)]))
    dataframe = pd.DataFrame(
        data=results[i], index=[
            "TX Frequency (MHz)", "RX Fundamental (MHz)", "SNR (dB)"])
    display(dataframe)

Channel: 2


,0,1,2,3,4,5,6,7,8,9
TX Frequency (MHz),200.000000,600.000000,1000.000000,1600.000000,2600.000000,3200.00000,3600.000000,4200.00000,4400.000000,4800.000000
RX Fundamental (MHz),200.000000,600.000000,1000.000000,1600.000000,2600.000000,3200.00000,3600.000000,4200.00000,4400.000000,4800.000000
SNR (dBFS),90.827538,90.737943,89.745502,89.032357,84.872278,82.26698,83.079912,81.38387,81.026402,83.291947


Channel: 3


,0,1,2,3,4,5,6,7,8,9
TX Frequency (MHz),200.000000,600.000000,1000.00000,1600.000000,2600.000000,3200.000000,3600.000000,4200.000000,4400.000000,4800.000000
RX Fundamental (MHz),200.000000,600.000000,1000.00000,1600.000000,2600.000000,3200.000000,3600.000000,4200.000000,4400.000000,4800.000000
SNR (dBFS),91.454898,91.012576,89.56917,89.157591,90.274721,90.375974,89.129994,86.672627,72.371455,87.099996


For the settings given in this notebook, a suitable SNR is any value
above 60 dB. The fundamental frequency of the received signal should be
exactly the same as the frequency of the transmitted tone.

## Conclusion <a class="anchor" id="conclusion"></a>
We presented a spectrum sweep application on RFSoC.
You have learned how to setup the spectrum sweep application
using loopback between a tone generator and spectrum analyzer
for each channel. The SNR and fundamental frequency was measured
for several candidate transmitter frequencies.

[⬅️ Spectrum Analysis on RFSoC](02_rf_spectrum_analysis.ipynb)

---
Copyright (C) 2022 Xilinx, Inc

SPDX-License-Identifier: BSD-3-Clause

----

----